## libraries

In [1]:
import os
import sys
from pathlib import Path
from pprint import pformat
from tempfile import TemporaryDirectory
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import geopandas as gpd
import pooch
import flopy
import flopy.plot
import flopy.utils
import rasterio
from rasterio.features import rasterize
from shapely.geometry import box
import contextily as cx
print(sys.version)
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {mpl.__version__}")
print(f"flopy version: {flopy.__version__}")

3.11.9 (main, Apr  2 2024, 13:43:44) [GCC 13.2.0]
numpy version: 1.26.3
matplotlib version: 3.9.2
flopy version: 3.9.2


## define simulation

In [2]:
# simulation setup
sim_name = 'rgtihm'
workspace = './model'
sim = flopy.mf6.MFSimulation(
    sim_name=sim_name,
    sim_ws=workspace,
    exe_name='mf6')
gwf = flopy.mf6.ModflowGwf(sim,
                           modelname=sim_name)
print("simulation and gwf packages created.")

simulation and gwf packages created.


## tdis

In [3]:
# tdis (temporal discretization) parameters
nper = 898
start_date = datetime(1940, 3, 1)
perioddata = []
for i in range(nper):
    month_start = start_date + timedelta(days=sum(p[0] for p in perioddata))
    if month_start.month == 12:
        month_end = datetime(month_start.year + 1, 1, 1)
    else:
        month_end = datetime(month_start.year, month_start.month + 1, 1)
    perlen = (month_end - month_start).days
    nstp = 2
    tsmult = 1.0
    perioddata.append((float(perlen), nstp, tsmult))

# create tdis
tdis = flopy.mf6.ModflowTdis(
    sim,
    time_units='days',
    nper=nper,
    perioddata=perioddata,
    start_date_time=start_date.strftime('%Y-%m-%d'),
)
print("tdis package created.")

tdis package created.


## dis

In [4]:
# tried feet units with meter crs, failed due to misalignment; tried rotation from lower-left, shifted grid left
# tried fixing top-left rotation, still misaligned; adopted anchoring rotation at top-left, matching qgis points
# our thinking: use exact qgis corners for precision, ensure grid aligns with real-world data via rasterio

# model crs for spatial reference
model_crs = 'epsg:26913'
xul = 251539.8073999998  # top-left x in meters, matches qgis
yul = 3639665.581800001  # top-left y in meters, matches qgis
angrot = 24.0  # rotation in degrees, matches qgis

# grid setup, all in meters
nlay = 9  # number of layers
nrow = 912  # number of rows
ncol = 328  # number of columns
delr = np.full(ncol, 660.0 * 0.3048)  # cell width in meters (660 ft to m)
delc = np.full(nrow, 660.0 * 0.3048)  # cell height in meters (660 ft to m)
# elevation data paths
top_file = '../owhm/model/2022/Data_Model_Arrays/layers/ElevFtDEMR.txt'
bot_files = [
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_RC2_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_USF1_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_USF2_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_MSF1_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_MSF2_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/TopLSF1_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_LSF2_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/TopBSMT_ft.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/BaseBSMT_ft.txt',
]

# load top elevation, convert feet to meters
try:
    top = np.loadtxt(top_file, dtype=float)
    if top.shape != (nrow, ncol):
        print(f"warning: top shape mismatch {top.shape}")
        top = top.reshape(nrow, ncol)[:nrow, :ncol]
    top = np.where(top == -99999, np.nan, top * 0.3048)  # ft to m
except Exception as e:
    print(f"error: loading top - {e}. exiting.")
    import sys
    sys.exit(1)

# load bottom elevations, convert feet to meters
botm = np.zeros((nlay, nrow, ncol), dtype=float)
for lay, bot_file in enumerate(bot_files):
    try:
        data = np.loadtxt(bot_file, dtype=float)
        if data.shape != (nrow, ncol):
            print(f"warning: bot shape mismatch {data.shape}")
            data = data.reshape(nrow, ncol)[:nrow, :ncol]
        botm[lay] = np.where(data == -99999, np.nan, data * 0.3048)  # ft to m
    except Exception as e:
        print(f"error: loading bot layer {lay} - {e}. exiting.")
        import sys
        sys.exit(1)

# load active area shapefile
shp_path = './shps/active_area.shp'
try:
    gdf = gpd.read_file(shp_path)
    if gdf.crs != model_crs:
        print(f"reprojecting shapefile crs")
        gdf = gdf.to_crs(model_crs)
    if len(gdf) != 1:
        print(f"warning: multiple features, using first")
    geometry = gdf.geometry.iloc[0]
except Exception as e:
    print(f"error: loading shapefile - {e}. exiting.")
    import sys
    sys.exit(1)

# set grid lower-left origin from qgis bottom-left
xll = 326161.8336  # lower-left x, from qgis bl
yll = 3472061.7670000014  # lower-left y, from qgis bl
theta = np.radians(angrot)

# grid coordinates, rotated to match qgis points
x = np.arange(ncol) * delr[0]
y = np.arange(nrow) * delc[0]
X, Y = np.meshgrid(x, y)
X_rot = xll + X * np.cos(theta) - Y * np.sin(theta)
Y_rot = yll + X * np.sin(theta) + Y * np.cos(theta)

# raster bounds in meters
minx, maxx = X_rot.min(), X_rot.max()
miny, maxy = Y_rot.min(), Y_rot.max()

# rasterize active area to define idomain
transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, ncol, nrow)
raster = rasterize([geometry], out_shape=(nrow, ncol), transform=transform, fill=0, default_value=1, dtype='int32')

# check raster coverage
print(f"active cells: {raster.sum()}/{nrow * ncol}")
print(f"grid bounds: {minx:.2f}, {maxx:.2f}, {miny:.2f}, {maxy:.2f}")
print(f"shapefile bounds: {gdf.total_bounds}")
if raster.sum() == 0:
    print("error: no active cells - check alignment")

# create 3d idomain for all layers
idomain = np.ones((nlay, nrow, ncol), dtype=int) * raster[np.newaxis, :, :]

# verify idomain
print(f"idomain active per layer: {[idomain[lay].sum() for lay in range(nlay)]}")

# method: used geometric transformation with rasterio to align grid and shapefile
# technique: anchored rotation at top-left, matched qgis points for precision
# reason: ensures perfect alignment with real-world data, avoiding misalignment
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
    idomain=idomain,
   # length_units=length_units,
    xorigin=xll,
    yorigin=yll,
    angrot=angrot,
)
print("dis set with idomain from shapefile, all layers \ndis package created.")

active cells: 52128/299136
grid bounds: 251621.63, 386256.62, 3472061.77, 3666237.73
shapefile bounds: [ 263314.4498 3490036.7519  357957.0726 3644537.2572]
idomain active per layer: [52128, 52128, 52128, 52128, 52128, 52128, 52128, 52128, 52128]
dis set with idomain from shapefile, all layers 
dis package created.


## oc 

In [5]:
# create oc (output control) package
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord=f'{sim_name}.cbc',
    head_filerecord=f'{sim_name}.hds',
    headprintrecord=[('COLUMNS', 10, 'WIDTH', 15, 'DIGITS', 6, 'GENERAL')],
    saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
    printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
)
print("oc package created.")

oc package created.


## ims

In [6]:
# iterative model solution (ims)
ims = flopy.mf6.ModflowIms(
    sim,
    pname='ims',
    complexity='simple',
    outer_dvclose=1e-4,
    outer_maximum=500,
    inner_maximum=100,
    inner_dvclose=1e-4,
    rcloserecord=0.001,
    linear_acceleration='cg',
    relaxation_factor=0.97,
)
print("ims package created.")

ims package created.


## ic

In [14]:
# grid dimensions (from dis)
nlay = 9
nrow = 912
ncol = 328

# head file paths (relative to rgtihm-main/model)
head_files = [
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY1.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY2.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY3.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY4.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY5.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY6.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY7.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY8.txt',
    '../owhm/model/2022/Data_Model_Arrays/layers/Initial_Head/WLCLY9.txt',
]

# read heads into 3d array, handle -999 as no data
strt = np.zeros((nlay, nrow, ncol), dtype=float)
for lay, head_file in enumerate(head_files):
    try:
        data = np.loadtxt(head_file, dtype=float)
        if data.shape != (nrow, ncol):
            print(f"warning: {head_file} shape {data.shape} != ({nrow}, {ncol})")
            data = data.reshape(nrow, ncol)[:nrow, :ncol]
        # handle -999 as NaN (not -99999)
        data = np.where(data == -999, np.nan, data)  # replace -999 with NaN
        strt[lay] = data
    except FileNotFoundError:
        print(f"error: {head_file} not found. IC requires this file. Exiting.")
        raise FileNotFoundError(f"Error: {head_file} not found.")
    except Exception as e:
        print(f"error reading {head_file}: {e}. Exiting.")
        raise Exception(f"Error reading {head_file}: {e}")

# fix strt for active cells (idomain == 1)
mask_active = idomain == 1  # 3D mask for active cells
for lay in range(nlay):
    active_mask = mask_active[lay]
    if np.isnan(strt[lay][active_mask]).any():
        # interpolate or assign default heads for active cells
        # use average of top and botm as default
        default_head = (top[active_mask] + botm[lay][active_mask]) / 2
        strt[lay][active_mask] = np.nan_to_num(strt[lay][active_mask], nan=default_head)
        # clip to ensure strt is within [botm, top]
        strt[lay][active_mask] = np.clip(strt[lay][active_mask], botm[lay][active_mask], top[active_mask])
    # ensure no NaN or Inf remain
    strt[lay] = np.nan_to_num(strt[lay], nan=np.nanmean(strt[lay][active_mask]), posinf=np.nanmean(strt[lay][active_mask]), neginf=np.nanmean(strt[lay][active_mask]))
    strt[lay][~active_mask] = np.nan  # keep inactive cells as NaN

# verify strt
print("NaN in strt fixed after interpolation:", np.isnan(strt).any())
for lay in range(nlay):
    strt_active = strt[lay][mask_active[lay]]
    print(f"Layer {lay} strt (active) min/max:", strt_active.min() if len(strt_active) > 0 else "No active cells", strt_active.max() if len(strt_active) > 0 else "No active cells")

# create ic package
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)
print("ic package created with -999 as no-data, idomain from shapefile, and interpolated active heads.")

NaN in strt fixed after interpolation: True
Layer 0 strt (active) min/max: 1133.921532 1223.290416
Layer 1 strt (active) min/max: 1129.3495320000002 1510.0294920000001
Layer 2 strt (active) min/max: 1134.0480240000002 1510.079784
Layer 3 strt (active) min/max: 1134.0480240000002 1510.079784
Layer 4 strt (active) min/max: 1134.0480240000002 1510.079784
Layer 5 strt (active) min/max: 1134.0480240000002 1510.079784
Layer 6 strt (active) min/max: 1134.0480240000002 1510.079784
Layer 7 strt (active) min/max: 1134.0480240000002 1548.356568
Layer 8 strt (active) min/max: 1134.0480240000002 1548.356568
ic package created with -999 as no-data, idomain from shapefile, and interpolated active heads.


## wel 

In [9]:
# wel package

#dir
well_folder = '../owhm/model/2022/Data_FeedFiles/WEL'

#read well files
well_files = [f for f in os.listdir(well_folder) if f.endswith('_FEED.txt')]
print(f"Found {len(well_files)} well files: {well_files}")

well_locations = []
flux_data = []
well_count = 0

for well_file in well_files:
    file_path = os.path.join(well_folder, well_file)
    with open(file_path, 'r') as f:
        lines = f.readlines()

    temp_locations = []
    temp_flux = []
    reading_locations = True

    for line in lines:
        if 'TEMPORAL INPUT' in line:
            reading_locations = False
            continue
        if reading_locations:
            if line.strip() and not line.startswith('#'):
                parts = line.split()
                layer = int(parts[0]) - 1
                row = int(parts[1]) - 1
                col = int(parts[2]) - 1
                well_name = parts[3].strip()
                temp_locations.append({'layer': layer, 'row': row, 'column': col, 'name': well_name})
        else:
            if line.strip() and not line.startswith('#'):
                parts = line.split()
                flux = []
                for x in parts:
                    if x.startswith('#'):
                        break
                    flux.append(float(x) if x.lower() != 'nan' else 0.0)
                if flux:
                    temp_flux.append(flux)

    # debug well names per file
    unique_names_in_file = set([loc['name'] for loc in temp_locations])
    print(f"File: {well_file}, Wells: {len(temp_locations)}, Unique Names: {unique_names_in_file}")
    
    well_locations.extend(temp_locations)
    if flux_data and len(temp_flux) > 0:
        for i in range(len(flux_data)):
            if i < len(temp_flux):
                flux_data[i].extend(temp_flux[i])
            else:
                flux_data[i].extend([0.0] * len(temp_locations))  # pad with zeros
        if len(flux_data) < len(temp_flux):
            for i in range(len(flux_data), len(temp_flux)):
                flux_data.append([0.0] * len(well_locations) + temp_flux[i])
    else:
        flux_data.extend(temp_flux)
    
    well_count += len(temp_locations)

print(f"Total Wells Read: {well_count}")
print(f"Total Flux Data Rows: {len(flux_data)}")
print(f"First 5 Wells: {well_locations[:5]}")
#print(f"First 5 Rows of Flux Data: {flux_data[:5]}")
unique_well_names = set([well['name'] for well in well_locations])
print(f"Combined Unique Well Names: {unique_well_names}")

# prepare stress period data
stress_period_data = {}
previous_flux = None

for sp in range(min(nper, len(flux_data))):
    current_flux = flux_data[sp]
    if sp == 0 or current_flux != previous_flux:
        stress_period_data[sp] = []
        for idx, well in enumerate(well_locations):
            if idx < len(current_flux) and current_flux[idx] != 0.0:
                well_tuple = ((well['layer'], well['row'], well['column']), current_flux[idx], well['name'])
                stress_period_data[sp].append(well_tuple)
       # print(f"Added {len(stress_period_data[sp])} wells to Stress Period {sp}")
    previous_flux = current_flux

if not stress_period_data:
    raise ValueError("Error: stress_period_data is empty!")
else:
    for sp in range(min(nper, 5)):
        if sp in stress_period_data:
            print(f"Stress Period {sp} Data: {stress_period_data[sp][:5]}") 

# create wel package
wel = flopy.mf6.ModflowGwfwel(
    gwf,
    stress_period_data=stress_period_data,
    save_flows=True,
    boundnames=True,
    print_input=True
)
print("WEL package created successfully.")
#max_wells = max(len(data) for data in stress_period_data.values() if data)
#print(f"Maximum wells in any stress period: {max_wells}")

Found 8 well files: ['MX_MnI_FEED.txt', 'NM_PDL_FEED.txt', 'NM_DOM_FEED.txt', 'NM_DOL_FEED.txt', 'NM_DCN_FEED.txt', 'NM_PDM_FEED.txt', 'TX_DOM_FEED.txt', 'NM_CLS_FEED.txt']
File: MX_MnI_FEED.txt, Wells: 33, Unique Names: {'WEL_MNI_MX'}
File: NM_PDL_FEED.txt, Wells: 73, Unique Names: {'WEL_DOM_NM'}
File: NM_DOM_FEED.txt, Wells: 7182, Unique Names: {'WEL_DOM_NM'}
File: NM_DOL_FEED.txt, Wells: 166, Unique Names: {'WEL_DOM_NM'}
File: NM_DCN_FEED.txt, Wells: 3, Unique Names: {'WEL_DOM_NM'}
File: NM_PDM_FEED.txt, Wells: 1308, Unique Names: {'WEL_DOM_NM'}
File: TX_DOM_FEED.txt, Wells: 60, Unique Names: {'WEL_DOM_TX'}
File: NM_CLS_FEED.txt, Wells: 85, Unique Names: {'WEL_DOM_NM'}
Total Wells Read: 8910
Total Flux Data Rows: 922
First 5 Wells: [{'layer': 2, 'row': 751, 'column': 186, 'name': 'WEL_MNI_MX'}, {'layer': 2, 'row': 748, 'column': 186, 'name': 'WEL_MNI_MX'}, {'layer': 2, 'row': 755, 'column': 186, 'name': 'WEL_MNI_MX'}, {'layer': 2, 'row': 805, 'column': 155, 'name': 'WEL_MNI_MX'}, {'

## npf

In [10]:
## node property flow package for k and k33
## in out case, horizontal and vertical conductivity - k11 and k22 is k, and k33 is conductivity of third ellipsoid axis - k33 tensor component. 
## turns out that k11 = k22 = k33 = k; since it is isotropic. we're using hk and vk.

# npf package
# zone file paths (relative to rgtihm-main/model)
zone_files = {
    'znlayrc': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/RC1_zonecode.txt',
    'znlay3': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/USF1_zonecode.txt',
    'znlay4': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/USF2_zonecode.txt',
    'znlay5': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/MSF1_zonecode.txt',
    'znlay6': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/MSF2_zonecode.txt',
    'znlay7': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/LSF1_zonecode.txt',
    'znlay8': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/LSF2_zonecode.txt',
    'znlay9': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/BSMT_zonecodeS.txt',
}

# read zone arrays
zone_arrays = {}
for name, filepath in zone_files.items():
    try:
        data = np.loadtxt(filepath, dtype=int)
        if data.shape != (nrow, ncol):
            print(f"warning: {filepath} shape {data.shape} != ({nrow}, {ncol})")
            data = data.reshape(nrow, ncol)[:nrow, :ncol]
        zone_arrays[name] = data
    except FileNotFoundError:
        print(f"warning: {filepath} not found, using default zone 1.")
        zone_arrays[name] = np.ones((nrow, ncol), dtype=int)
    except Exception as e:
        print(f"error reading {filepath}: {e}. using default zone 1.")
        zone_arrays[name] = np.ones((nrow, ncol), dtype=int)

# layer-to-zone mapping
layer_zones = {
    0: 'znlayrc', 1: 'znlayrc', 2: 'znlay3', 3: 'znlay4',
    4: 'znlay5', 5: 'znlay6', 6: 'znlay7', 7: 'znlay8', 8: 'znlay9'
}

# complete pval_data from RGTIHM.PVL for hydraulic conductivity
pval_data = {
    'l1hk10': 100.0, 'l1hk20': 30.0, 'l1hk30': 0.2629016, 'l1hk51': 0.14489073,
    'l2hk10': 100.0, 'l2hk20': 30.0, 'l2hk30': 0.2629016, 'l2hk51': 0.14489073,
    'l3hk15': 100.0, 'l3hk25': 100.0, 'l3hk35': 9.2195476, 'l3hk50': 30.0, 'l3hk51': 6.4479888,
    'l3hk55': 1.0, 'l3hk60': 0.17476377, 'l3hk90': 0.01,
    'l4hk15': 100.0, 'l4hk25': 100.0, 'l4hk35': 9.2195476, 'l4hk50': 30.0, 'l4hk51': 6.4479888,
    'l4hk55': 1.0, 'l4hk60': 0.17476377, 'l4hk90': 0.01,
    'l5hk35': 30.0, 'l5hk50': 0.20053775, 'l5hk55': 0.66872867, 'l5hk65': 1.0, 'l5hk90': 0.016224702,
    'l5hk100': 0.035072631,
    'l6hk25': 100.0, 'l6hk35': 30.0, 'l6hk40': 3.9352467, 'l6hk50': 0.20053775, 'l6hk55': 0.66872867,
    'l6hk65': 1.0, 'l6hk90': 0.016224702, 'l6hk100': 0.035072631,
    'l7hk35': 1.0, 'l7hk40': 2.1805362, 'l7hk55': 0.1, 'l7hk65': 0.38348569, 'l7hk90': 0.1, 'l7hk100': 0.023773051,
    'l8hk35': 1.0, 'l8hk40': 2.1805362, 'l8hk55': 0.1, 'l8hk65': 0.38348569, 'l8hk90': 0.1, 'l8hk100': 0.023773051,
    'l9hk11': 0.008606693, 'l9hk21': 0.58382645, 'l9hk31': 0.0042014914, 'l9hk41': 0.17529754, 'l9hk56': 10.0,
    'l9hk61': 0.048328901, 'l9hk71': 3.2559841,
    # vertical K (VK) for K33
    'l1vk10': 1.091422, 'l1vk20': 1.2296545, 'l1vk30': 155.00641, 'l1vk51': 1003.4731,
    'l2vk10': 1.091422, 'l2vk20': 1.2296545, 'l2vk30': 155.00641, 'l2vk51': 1003.4731,
    'l3vk15': 1.0, 'l3vk25': 10.413017, 'l3vk35': 5.2372343, 'l3vk50': 6.9875221, 'l3vk51': 8.741806,
    'l3vk55': 1358.6343, 'l3vk60': 1.0, 'l3vk90': 9.1188492,
    'l4vk15': 1.0, 'l4vk25': 10.413017, 'l4vk35': 5.2372343, 'l4vk50': 6.9875221, 'l4vk51': 8.741806,
    'l4vk55': 1358.6343, 'l4vk60': 1.0, 'l4vk90': 9.1188492,
    'l5vk35': 1.0, 'l5vk50': 5845.6619, 'l5vk55': 11.489684, 'l5vk65': 17.306335, 'l5vk90': 112.8303,
    'l5vk100': 10.288813,
    'l6vk25': 4.4199196, 'l6vk35': 1.0, 'l6vk40': 247.36765, 'l6vk50': 5845.6619, 'l6vk55': 11.489684,
    'l6vk65': 17.306335, 'l6vk90': 112.8303, 'l6vk100': 10.288813,
    'l7vk35': 2.7818614, 'l7vk40': 2.3683035, 'l7vk55': 1.2711653, 'l7vk65': 11.44632, 'l7vk90': 1.0,
    'l7vk100': 597.30784,
    'l8vk35': 2.7818614, 'l8vk40': 2.3683035, 'l8vk55': 1.2711653, 'l8vk65': 11.44632, 'l8vk90': 1.0,
    'l8vk100': 597.30784,
    'l9vk11': 1.0, 'l9vk21': 48.550788, 'l9vk31': 2.7429583, 'l9vk41': 1.9085057, 'l9vk56': 29.289344,
    'l9vk61': 1.0, 'l9vk71': 10000.0,
    # K22 not explicitly in PVL, assume isotropic (k22 = hk)
}

# initialize arrays with no-data handling
hk = np.ones((nlay, nrow, ncol), dtype=float) * 1e-10  # default K > 0
k22 = hk.copy()  # y-direction K, defaults to hk (isotropic)
k33 = hk.copy()  # vertical K, defaults to hk if no anisotropy
icelltype = [0] * nlay  # confined by default, adjust if needed

# populate hk, k22, k33, handle -99999 as no data
for param, value in pval_data.items():
    lay = int(param[1]) - 1
    iz = int(param.split('hk' if 'hk' in param else 'vk')[1]) if 'hk' in param or 'vk' in param else 1
    zone_array = zone_arrays[layer_zones[lay]]
    mask = (zone_array == iz)
    if value == -99999:  # treat -99999 as no data
        hk[lay][mask] = np.nan
        if 'vk' in param:
            k33[lay][mask] = np.nan
        else:
            k22[lay][mask] = np.nan
    else:
        if 'hk' in param:
            hk[lay][mask] = np.maximum(hk[lay][mask], value)  # ensure K > 0, use np.maximum for arrays
            k22[lay][mask] = np.maximum(k22[lay][mask], value)  # assume isotropic unless K22 specified
        elif 'vk' in param:
            k33[lay][mask] = np.maximum(k33[lay][mask], value)  # vertical K
            hk[lay][mask] = np.maximum(hk[lay][mask], 1e-10)  # ensure horizontal K > 0

# update to match idomain (inactive cells from DIS)
idomain = dis.idomain.array  # from dis package
hk[idomain == 0] = np.nan  # set inactive cells to NaN
k22[idomain == 0] = np.nan
k33[idomain == 0] = np.nan

# create npf with optional features
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_flows=True,
    icelltype=icelltype,
    k=hk,
    k22=k22,
    k33=k33,
    save_specific_discharge=True,
    alternative_cell_averaging="harmonic",  # match MF-OWHM’s likely averaging
    # xt3d=True  # uncomment if 3D anisotropy is needed
)
print("npf package created from zonecode files and PVL/RGTIHM.PVL")


npf package created from zonecode files and PVL/RGTIHM.PVL


## sto

In [11]:
# sto package
# multiplier and thickness file paths (relative to rgtihm-main/model)
mult_files = {
    'uc_rc': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/RC1_UC.txt',
    'uc_usf1': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/USF1_UC.txt',
    'cf_usf1': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/USF1_CF.txt',
    'uc_msf1': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/MSF1_UC.txt',
    'cf_msf1': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/MSF1_CF.txt',
    'uc_lsf1': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/LSF1_UC.txt',
    'cf_lsf1': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/LSF1_CF.txt',
    'uc_bd': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/BSMT_UC.txt',
    'cf_bd': '../owhm/model/2022/Data_Model_Arrays/layers/CF_UC/BSMT_CF.txt',
    'thk_rc1': '../owhm/model/2022/Data_Model_Arrays/layers/Geometry/THK_RC1_ft.txt',
    'thk_usf1': '../owhm/model/2022/Data_Model_Arrays/layers/Geometry/THK_USF1_ft.txt',
    'thk_msf1': '../owhm/model/2022/Data_Model_Arrays/layers/Geometry/THK_MSF1_ft.txt',
    'thk_lsf1': '../owhm/model/2022/Data_Model_Arrays/layers/Geometry/THK_LSF1_ft.txt',
    'thk_bd': '../owhm/model/2022/Data_Model_Arrays/layers/Geometry/THK_BSMT_ft.txt',
}

# read multiplier and thickness arrays, handle -99999 as no data
mult_arrays = {}
for name, filepath in mult_files.items():
    try:
        data = np.loadtxt(filepath, dtype=float)
        if data.shape != (nrow, ncol):
            print(f"warning: {filepath} shape {data.shape} != ({nrow}, {ncol})")
            data = data.reshape(nrow, ncol)[:nrow, :ncol]
        data = np.where(data == -99999, np.nan, data * 0.3048 if 'thk' in name else data)  # convert thk ft to m
        mult_arrays[name] = data
    except FileNotFoundError:
        print(f"warning: {filepath} not found, using default 1.0.")
        mult_arrays[name] = np.ones((nrow, ncol), dtype=float)
    except Exception as e:
        print(f"error reading {filepath}: {e}. using default 1.0.")
        mult_arrays[name] = np.ones((nrow, ncol), dtype=float)

# constants from mul
ss_base = 0.000001  # default specific storage (1/m)
sy_base = 0.30  # default specific yield
phi = {'rc': 0.25, 'us': 0.22, 'ms': 0.12, 'ls': 0.08, 'bd': 0.07}  # porosity by layer group
comp_h2o = 1.432195e-06  # water compressibility (1/m)
offset = 0.0001  # small offset for thickness to avoid division by zero

# zone file paths (relative to rgtihm-main/model, from npf)
zone_files = {
    'znlayrc': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/RC1_zonecode.txt',
    'znlay3': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/USF1_zonecode.txt',
    'znlay4': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/USF2_zonecode.txt',
    'znlay5': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/MSF1_zonecode.txt',
    'znlay6': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/MSF2_zonecode.txt',
    'znlay7': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/LSF1_zonecode.txt',
    'znlay8': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/LSF2_zonecode.txt',
    'znlay9': '../owhm/model/2022/Data_Model_Arrays/layers/El_IBD_Zone/BSMT_zonecodeS.txt',
}

# read zone arrays (reuse from npf if already loaded, or load here)
zone_arrays = {}
for name, filepath in zone_files.items():
    if name not in zone_arrays:  # avoid reloading if npf already loaded
        try:
            data = np.loadtxt(filepath, dtype=int)
            if data.shape != (nrow, ncol):
                print(f"warning: {filepath} shape {data.shape} != ({nrow}, {ncol})")
                data = data.reshape(nrow, ncol)[:nrow, :ncol]
            zone_arrays[name] = data
        except FileNotFoundError:
            print(f"warning: {filepath} not found, using default zone 1.")
            zone_arrays[name] = np.ones((nrow, ncol), dtype=int)
        except Exception as e:
            print(f"error reading {filepath}: {e}. using default zone 1.")
            zone_arrays[name] = np.ones((nrow, ncol), dtype=int)

# layer-to-zone mapping (from npf)
layer_zones = {
    0: 'znlayrc', 1: 'znlayrc', 2: 'znlay3', 3: 'znlay4',
    4: 'znlay5', 5: 'znlay6', 6: 'znlay7', 7: 'znlay8', 8: 'znlay9'
}

# complete pval_data from RGTIHM.PVL for storage
pval_data = {
    'l2ss10': 5.0, 'l2ss20': 25.0, 'l2ss30': 8.2691995, 'l2ss51': 8.758604,
    'l3ss15': 1.0955468, 'l3ss25': 2.9409115, 'l3ss35': 3.33, 'l3ss50': 1.069701, 'l3ss51': 2.0, 'l3ss55': 10.0,
    'l3ss60': 5.1091253, 'l3ss90': 6.5520935,
    'l4ss15': 1.0955468, 'l4ss25': 2.9409115, 'l4ss35': 3.33, 'l4ss50': 1.069701, 'l4ss51': 2.0, 'l4ss55': 10.0,
    'l4ss60': 5.1091253, 'l4ss90': 6.5520935,
    'l5ss35': 0.4, 'l5ss50': 0.80498139, 'l5ss55': 1.104017, 'l5ss65': 0.5723319, 'l5ss90': 0.71704178,
    'l5ss100': 1.67,
    'l6ss25': 1.67, 'l6ss35': 0.4, 'l6ss40': 0.95152044, 'l6ss50': 0.80498139, 'l6ss55': 1.104017, 'l6ss65': 0.5723319,
    'l6ss90': 0.71704178, 'l6ss100': 1.67,
    'l7ss35': 1.2993514, 'l7ss40': 2.33, 'l7ss55': 2.33, 'l7ss65': 2.050916, 'l7ss90': 0.91998654, 'l7ss100': 1.4320346,
    'l8ss35': 1.2993514, 'l8ss40': 2.33, 'l8ss55': 2.33, 'l8ss65': 2.050916, 'l8ss90': 0.91998654, 'l8ss100': 1.4320346,
    'l9ss11': 2.33, 'l9ss21': 2.33, 'l9ss31': 0.88759879, 'l9ss41': 0.91689225, 'l9ss56': 0.667, 'l9ss61': 1.8936826,
    'l9ss71': 2.33,
    'l1sy10': 1.0, 'l1sy20': 1.0, 'l1sy30': 0.825, 'l1sy51': 1.0,
    'l3sy15': 0.50864162, 'l3sy25': 0.87375378, 'l3sy35': 0.59566227, 'l3sy50': 0.553, 'l3sy51': 0.29098953,
    'l3sy55': 0.73273826, 'l3sy60': 0.37109912, 'l3sy90': 0.2,
    'l5sy35': 0.536, 'l5sy50': 0.83810284, 'l5sy55': 1.0, 'l5sy65': 1.0, 'l5sy90': 0.51269604, 'l5sy100': 0.333,
    'l7sy35': 0.59566227, 'l7sy40': 0.88904665, 'l7sy55': 0.64751193, 'l7sy65': 0.83651446, 'l7sy90': 0.44452383,
    'l7sy100': 0.44452383,
    'l9sy11': 0.74220455, 'l9sy21': 0.333, 'l9sy31': 0.89490974, 'l9sy41': 0.87571372, 'l9sy56': 0.39531322,
    'l9sy61': 1.0, 'l9sy71': 1.0,
}

# initialize arrays with no-data handling
ss = np.ones((nlay, nrow, ncol), dtype=float) * ss_base  # default specific storage (1/m)
sy = np.ones((nlay, nrow, ncol), dtype=float) * sy_base  # default specific yield
iconvert = [0] * nlay  # confined by default, adjust if needed

# populate ss and sy with PVL values, handle -99999 and multipliers
for param, value in pval_data.items():
    lay = int(param[1]) - 1
    iz = int(param.split('ss' if 'ss' in param else 'sy')[1]) if 'ss' in param or 'sy' in param else 1
    zone_array = zone_arrays[layer_zones[lay]]
    mask = (zone_array == iz)
    if value == -99999:  # treat -99999 as no data
        ss[lay][mask] = np.nan
        sy[lay][mask] = np.nan
    else:
        if 'ss' in param:
            ss[lay][mask] = np.maximum(ss[lay][mask], value)  # ensure SS > 0, use np.maximum
        elif 'sy' in param:
            sy[lay][mask] = np.clip(value, 0.0, 1.0)  # ensure 0 < SY < 1

# apply multipliers and thickness, handle NaN
for lay in range(nlay):
    zone_array = zone_arrays[layer_zones[lay]]
    if lay in [0, 1]:  # rc
        thk = mult_arrays['thk_rc1']
        uc = mult_arrays['uc_rc']
        ss[lay] = np.where(np.isnan(ss[lay]) | np.isnan(uc) | np.isnan(thk), np.nan,
                           np.maximum(ss[lay], (phi['rc'] * comp_h2o + ss_base) * uc))
        sy[lay] = np.where(np.isnan(sy[lay]) | np.isnan(uc) | np.isnan(thk), np.nan,
                           np.clip((sy_base / (thk + offset) + phi['rc'] * comp_h2o) * uc, 0.0, 1.0))
    elif lay in [2, 3]:  # usf
        thk = mult_arrays['thk_usf1']
        uc = mult_arrays['uc_usf1']
        cf = mult_arrays['cf_usf1']
        ss[lay] = np.where(np.isnan(ss[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.maximum(ss[lay], (phi['us'] * comp_h2o + ss_base) * (cf + uc) if lay == 3 else (phi['us'] * comp_h2o) * cf + ss_base * cf))
        sy[lay] = np.where(np.isnan(sy[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.clip((sy_base / (thk + offset) + phi['us'] * comp_h2o) * uc, 0.0, 1.0))
    elif lay in [4, 5]:  # msf
        thk = mult_arrays['thk_msf1']
        uc = mult_arrays['uc_msf1']
        cf = mult_arrays['cf_msf1']
        ss[lay] = np.where(np.isnan(ss[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.maximum(ss[lay], (phi['ms'] * comp_h2o + ss_base) * (cf + uc) if lay == 5 else (phi['ms'] * comp_h2o) * cf + ss_base * cf))
        sy[lay] = np.where(np.isnan(sy[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.clip((sy_base / (thk + offset) + phi['ms'] * comp_h2o) * uc, 0.0, 1.0))
    elif lay in [6, 7]:  # lsf
        thk = mult_arrays['thk_lsf1']
        uc = mult_arrays['uc_lsf1']
        cf = mult_arrays['cf_lsf1']
        ss[lay] = np.where(np.isnan(ss[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.maximum(ss[lay], (phi['ls'] * comp_h2o + ss_base) * (cf + uc) if lay == 7 else (phi['ls'] * comp_h2o) * cf + ss_base * cf))
        sy[lay] = np.where(np.isnan(sy[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.clip((sy_base / (thk + offset) + phi['ls'] * comp_h2o) * uc, 0.0, 1.0))
    elif lay == 8:  # bd
        thk = mult_arrays['thk_bd']
        uc = mult_arrays['uc_bd']
        cf = mult_arrays['cf_bd']
        ss[lay] = np.where(np.isnan(ss[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.maximum(ss[lay], (phi['bd'] * comp_h2o + ss_base) * (cf + uc)))
        sy[lay] = np.where(np.isnan(sy[lay]) | np.isnan(uc) | np.isnan(cf) | np.isnan(thk), np.nan,
                           np.clip((sy_base / (thk + offset) + phi['bd'] * comp_h2o) * uc, 0.0, 1.0))

# update ss and sy to match idomain (inactive cells from DIS)
idomain = dis.idomain.array  # from dis package
ss[idomain == 0] = np.nan  # set inactive cells to NaN
sy[idomain == 0] = np.nan

# create sto
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    save_flows=True,
    iconvert=iconvert,
    ss=ss,
    sy=sy,
    transient=True,
)
print("sto package created")

sto package created


In [38]:
# hfb, wel, maw, sfr, ghb, et, 

In [13]:
# create 3D mask for active cells (idomain == 1) across all layers
mask_active = idomain == 1  # shape (nlay, nrow, ncol)

# check and fix NaN/invalid values in NPF
hk = np.nan_to_num(hk, nan=1e-10, posinf=1e-10, neginf=1e-10)  # ensure K > 0
k22 = np.nan_to_num(k22, nan=1e-10, posinf=1e-10, neginf=1e-10)
k33 = np.nan_to_num(k33, nan=1e-10, posinf=1e-10, neginf=1e-10)
print("NaN in hk fixed:", np.isnan(hk).any())
for lay in range(nlay):
    hk_active = hk[lay][mask_active[lay]]  # apply mask per layer
    print(f"Layer {lay} hk (active) min/max:", hk_active.min() if len(hk_active) > 0 else "No active cells", hk_active.max() if len(hk_active) > 0 else "No active cells")

# check and fix NaN/invalid values in STO
ss = np.nan_to_num(ss, nan=ss_base, posinf=ss_base, neginf=ss_base)  # ensure SS > 0
sy = np.nan_to_num(sy, nan=sy_base, posinf=sy_base, neginf=sy_base)  # ensure 0 < SY < 1
sy = np.clip(sy, 0.0, 1.0)
print("NaN in ss fixed:", np.isnan(ss).any())
for lay in range(nlay):
    ss_active = ss[lay][mask_active[lay]]
    sy_active = sy[lay][mask_active[lay]]
    print(f"Layer {lay} ss (active) min/max:", ss_active.min() if len(ss_active) > 0 else "No active cells", ss_active.max() if len(ss_active) > 0 else "No active cells")
    print(f"Layer {lay} sy (active) min/max:", sy_active.min() if len(sy_active) > 0 else "No active cells", sy_active.max() if len(sy_active) > 0 else "No active cells")

# check and fix NaN/invalid values in IC
strt = np.nan_to_num(strt, nan=np.nanmean(strt[mask_active]), posinf=np.nanmean(strt[mask_active]), neginf=np.nanmean(strt[mask_active]))
for lay in range(nlay):
    mask_active_lay = mask_active[lay]
    strt[lay][~mask_active_lay] = np.nan  # keep inactive as NaN
    strt[lay][mask_active_lay] = np.clip(strt[lay][mask_active_lay], botm[lay][mask_active_lay], top[mask_active_lay])
print("NaN in strt fixed:", np.isnan(strt).any())
for lay in range(nlay):
    strt_active = strt[lay][mask_active[lay]]
    print(f"Layer {lay} strt (active) min/max:", strt_active.min() if len(strt_active) > 0 else "No active cells", strt_active.max() if len(strt_active) > 0 else "No active cells")

# check WEL fluxes (already verified in your output, but add for completeness)
fluxes = [item[1] for sp_data in wel.stress_period_data.values() for item in sp_data if sp_data]
print("WEL fluxes min/max:", min(fluxes) if fluxes else "No fluxes", max(fluxes) if fluxes else "No fluxes")
print("NaN in WEL fluxes:", any(np.isnan(fluxes)) if fluxes else False)

# update packages if needed (e.g., NPF, STO, IC)
npf.k = hk
npf.k22 = k22
npf.k33 = k33
sto.ss = ss
sto.sy = sy
ic.strt = strt

NaN in hk fixed: False
Layer 0 hk (active) min/max: 1e-10 100.0
Layer 1 hk (active) min/max: 1e-10 100.0
Layer 2 hk (active) min/max: 1e-10 100.0
Layer 3 hk (active) min/max: 1e-10 100.0
Layer 4 hk (active) min/max: 1e-10 30.0
Layer 5 hk (active) min/max: 1e-10 100.0
Layer 6 hk (active) min/max: 1e-10 2.1805362
Layer 7 hk (active) min/max: 1e-10 2.1805362
Layer 8 hk (active) min/max: 0.0042014914 10.0
NaN in ss fixed: False
Layer 0 ss (active) min/max: 1e-06 1.35804875e-06
Layer 0 sy (active) min/max: 0.0 0.019685268252751284
Layer 1 ss (active) min/max: 1e-06 25.0
Layer 1 sy (active) min/max: 0.0 0.019685268252751284
Layer 2 ss (active) min/max: 1e-06 10.0
Layer 2 sy (active) min/max: 0.0 0.4920455832475711
Layer 3 ss (active) min/max: 1e-06 10.0
Layer 3 sy (active) min/max: 0.0 0.4920455832475711
Layer 4 ss (active) min/max: 1e-06 1.104017
Layer 4 sy (active) min/max: 0.0 0.0
Layer 5 ss (active) min/max: 1e-06 1.67
Layer 5 sy (active) min/max: 0.0 0.0
Layer 6 ss (active) min/max: 1e-

AttributeError: 'MFPandasTransientList' object has no attribute 'values'

## write and run

In [15]:
# write simulation
sim.write_simulation()

# optionally run simulation (uncomment to test)
# sim.run_simulation()

print("simulation written to ./model directory. Check for errors.")

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims...
  writing model rgtihm...
    writing model name file...
    writing package dis...
    writing package oc...
    writing package wel_0...
    writing package npf...
    writing package sto...
    writing package ic...
simulation written to ./model directory. Check for errors.


## Visualization

In [ ]:
# grid coords in meters (from dis)
xll = dis.xorigin.get_data()
yll = dis.yorigin.get_data()
angrot = dis.angrot.get_data()
delr = dis.delr.get_data()
delc = dis.delc.get_data()
x = np.arange(ncol) * delr[0]
y = np.arange(nrow) * delc[0]
X, Y = np.meshgrid(x, y)
X_rot = xll + X * np.cos(np.radians(angrot)) - Y * np.sin(np.radians(angrot))
Y_rot = yll + X * np.sin(np.radians(angrot)) + Y * np.cos(np.radians(angrot))

# 1. overall model domain with idomain and shapefile
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(idomain[0], extent=[X_rot.min(), X_rot.max(), Y_rot.min(), Y_rot.max()], cmap='binary', alpha=0.5)
gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)
cx.add_basemap(ax, crs=model_crs, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_title('model domain with idomain (layer 1) and active_area.shp')
ax.set_xlabel('easting (m)')
ax.set_ylabel('northing (m)')
plt.tight_layout()
plt.show()

# 2. grid outline with idomain
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.cm.colors.ListedColormap(['white', 'lightgrey'])
bounds = [0, 1, 2]
norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
ax.imshow(idomain[0], extent=[X_rot.min(), X_rot.max(), Y_rot.min(), Y_rot.max()], cmap=cmap, norm=norm, alpha=0.8, interpolation='nearest')
gdf.plot(ax=ax, edgecolor='lightgrey', facecolor='none', linewidth=2, alpha=0.5)
for i in range(nrow):
    ax.plot([X_rot[i, 0], X_rot[i, -1]], [Y_rot[i, 0], Y_rot[i, -1]], 'k-', lw=0.5)
for j in range(ncol):
    ax.plot([X_rot[0, j], X_rot[-1, j]], [Y_rot[0, j], Y_rot[-1, j]], 'k-', lw=0.5)
cx.add_basemap(ax, crs=model_crs, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_title('grid with idomain (layer 1)')
ax.set_xlabel('easting (m)')
ax.set_ylabel('northing (m)')
plt.tight_layout()
plt.show()


In [ ]:
# cross-sections for rows 538, 741, 103, and column 75
rows = [538, 741, 103]
col = 75

# Corrected geological layer names (only 9 layers, not 10)
layer_names = ["Top RC1", "Top RC2", "Top USF1", "Top USF2", 
               "Top MSF1", "Top MSF2", "Top LSF1", "Top LSF2", 
               "Top BSMT"]  # Removed "Base BSMT" since it's botm[-1]

for row in rows:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6), gridspec_kw={'width_ratios': [3, 1]})

    # elevation profile - now plotting geological layers for all cells, ignoring idomain
    for lay, name in zip(range(nlay), layer_names):
        bot = botm[lay, row, :]
        ax1.plot(X_rot[row, :], bot, label=name)

    ax1.plot(X_rot[row, :], top[row, :], 'k-', label='Elev')

    # Check and print inactive cells per layer
    inactive_cells = [(name, (idomain[lay, row, :] == 0).sum()) for lay, name in enumerate(layer_names)]
    print(f"Inactive cells in row {row}: {inactive_cells}")

    # Previously shaded active cells, now showing full geological layers
    ax1.fill_between(X_rot[row, :], botm[-1, row, :], top[row, :], color='gray', alpha=0.3)

    ax1.set_title(f'cross-section (row {row})')
    ax1.set_xlabel('easting (m)')
    ax1.set_ylabel('elevation (m)')
    ax1.legend()

    # basemap inset for location
    ax2.imshow(idomain[0], extent=[X_rot.min(), X_rot.max(), Y_rot.min(), Y_rot.max()], cmap='binary', alpha=0.5)
    gdf.plot(ax=ax2, edgecolor='red', facecolor='none', linewidth=2)
    cx.add_basemap(ax2, crs=model_crs, source=cx.providers.OpenStreetMap.Mapnik)
    ax2.plot(X_rot[row, :], Y_rot[row, :], 'b-', lw=2, label='cross-section line')
    ax2.set_title('location map')
    ax2.legend()
    ax2.set_xticks([])
    ax2.set_yticks([])
    plt.tight_layout()
    plt.show()

# cross-section for column 75
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6), gridspec_kw={'width_ratios': [3, 1]})

for lay, name in zip(range(nlay), layer_names):
    bot = botm[lay, :, col]
    ax1.plot(Y_rot[:, col], bot, label=name)

ax1.plot(Y_rot[:, col], top[:, col], 'k-', label='Elev')

# Check and print inactive cells in the column
inactive_cells_col = [(name, (idomain[lay, :, col] == 0).sum()) for lay, name in enumerate(layer_names)]
print(f"Inactive cells in column {col}: {inactive_cells_col}")

ax1.fill_between(Y_rot[:, col], botm[-1, :, col], top[:, col], color='gray', alpha=0.3)
ax1.set_title(f'cross-section (column {col})')
ax1.set_xlabel('northing (m)')
ax1.set_ylabel('elevation (m)')
ax1.legend()

ax2.imshow(idomain[0], extent=[X_rot.min(), X_rot.max(), Y_rot.min(), Y_rot.max()], cmap='binary', alpha=0.5)
gdf.plot(ax=ax2, edgecolor='red', facecolor='none', linewidth=2)
cx.add_basemap(ax2, crs=model_crs, source=cx.providers.OpenStreetMap.Mapnik)
ax2.plot(X_rot[:, col], Y_rot[:, col], 'b-', lw=2, label='cross-section line')
ax2.set_title('location map')
ax2.legend()
ax2.set_xticks([])
ax2.set_yticks([])
plt.tight_layout()
plt.show()
